***GENERATED CODE FOR clas PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'name', 'transformation_label': 'String Indexer'}], 'feature': 'name', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '500', 'mean': '123123.0', 'stddev': 'nan', 'min': '!5 min frm MTR!Cozy 2 BDR @ the heart of KLN - TST', 'max': 'ï¼\x81NEWï¼\x81 #STUDIO@JORDAN#3MINS MTR#8MINS AIRPORT TRAIN', 'missing': '0', 'distinct': '496'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'name'}, {'feature_label': 'name', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('name')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'host_id', 'transformation_label': 'String Indexer'}], 'feature': 'host_id', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '95783051.5', 'stddev': '91552205.01', 'min': '101777143', 'max': '98992897', 'missing': '0', 'distinct': '350'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'host_id'}, {'feature_label': 'host_id', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('host_id')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'host_since', 'transformation_label': 'String Indexer'}], 'feature': 'host_since', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '42429.52', 'stddev': '782.94', 'min': '40222', 'max': '44189', 'missing': '0', 'distinct': '330'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'host_since'}, {'feature_label': 'host_since', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('host_since')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'host_response_time', 'transformation_label': 'String Indexer'}], 'feature': 'host_response_time', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '382', 'mean': '', 'stddev': '', 'min': 'a few days or more', 'max': 'within an hour', 'missing': '118', 'distinct': '4'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'host_response_time'}, {'feature_label': 'host_response_time', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('host_response_time')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'host_acceptance_rate', 'transformation_label': 'String Indexer'}], 'feature': 'host_acceptance_rate', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '342', 'mean': '0.7', 'stddev': '0.36', 'min': '0', 'max': '1', 'missing': '158', 'distinct': '47'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'host_acceptance_rate'}, {'feature_label': 'host_acceptance_rate', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('host_acceptance_rate')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'host_is_superhost', 'transformation_label': 'String Indexer'}], 'feature': 'host_is_superhost', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'f', 'max': 't', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'host_is_superhost'}, {'feature_label': 'host_is_superhost', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('host_is_superhost')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'host_has_profile_pic', 'transformation_label': 'String Indexer'}], 'feature': 'host_has_profile_pic', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'f', 'max': 't', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'host_has_profile_pic'}, {'feature_label': 'host_has_profile_pic', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('host_has_profile_pic')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'host_identity_verified', 'transformation_label': 'String Indexer'}], 'feature': 'host_identity_verified', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'f', 'max': 't', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'host_identity_verified'}, {'feature_label': 'host_identity_verified', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('host_identity_verified')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'neighbourhood', 'transformation_label': 'String Indexer'}], 'feature': 'neighbourhood', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Central & Western', 'max': 'Yuen Long', 'missing': '0', 'distinct': '17'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'neighbourhood'}, {'feature_label': 'neighbourhood', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('neighbourhood')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'district', 'transformation_label': 'String Indexer'}], 'feature': 'district', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '0', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '500', 'distinct': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'district'}, {'feature_label': 'district', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('district')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'city', 'transformation_label': 'String Indexer'}], 'feature': 'city', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Hong Kong', 'max': 'Hong Kong', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'city'}, {'feature_label': 'city', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('city')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'longitude', 'transformation_label': 'String Indexer'}], 'feature': 'longitude', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '114.15', 'stddev': '0.05', 'min': '113.91289', 'max': '114.27257', 'missing': '0', 'distinct': '469'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'longitude'}, {'feature_label': 'longitude', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('longitude')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'property_type', 'transformation_label': 'String Indexer'}], 'feature': 'property_type', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Entire apartment', 'max': 'Entire apartment', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'property_type'}, {'feature_label': 'property_type', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('property_type')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'room_type', 'transformation_label': 'String Indexer'}], 'feature': 'room_type', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Entire place', 'max': 'Entire place', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'room_type'}, {'feature_label': 'room_type', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('room_type')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'accommodates', 'transformation_label': 'String Indexer'}], 'feature': 'accommodates', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '3.51', 'stddev': '2.41', 'min': '1', 'max': '9', 'missing': '0', 'distinct': '15'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'accommodates'}, {'feature_label': 'accommodates', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('accommodates')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'bedrooms', 'transformation_label': 'String Indexer'}], 'feature': 'bedrooms', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '313', 'mean': '1.73', 'stddev': '0.98', 'min': '1', 'max': '5', 'missing': '187', 'distinct': '6'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'bedrooms'}, {'feature_label': 'bedrooms', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('bedrooms')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'instant_bookable', 'transformation_label': 'String Indexer'}], 'feature': 'instant_bookable', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'f', 'max': 't', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'instant_bookable'}, {'feature_label': 'instant_bookable', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('instant_bookable')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Years until 2021', 'transformation_label': 'String Indexer'}], 'feature': 'Years until 2021', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '5.35', 'stddev': '2.19', 'min': '1', 'max': '9', 'missing': '0', 'distinct': '11'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Years until 2021'}, {'feature_label': 'Years until 2021', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Years until 2021')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Recommended Properties', 'transformation_label': 'String Indexer'}], 'feature': 'Recommended Properties', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Average', 'max': 'NotRecommended', 'missing': '0', 'distinct': '4'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Recommended Properties'}, {'feature_label': 'Recommended Properties', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Recommended Properties')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pyspark


def functionClassification(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTClassifier(verbosity=3, n_jobs=-1, generations=10, max_time_mins=5,
                               population_size=15, use_dask=True)
    tpotModel.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run clasHooks.ipynb
try:
	#sourcePreExecutionHook()

	hkv = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/HK_v1.csv', 'filename': 'HK_v1.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line', 'domain': 'http://172.31.59.158', 'port': '40070', 'dirPath': '/FileStore/platform', 'server_url': '/nexusMax/NexusMaxPlatform/uploads/platform/'}")
	#sourcePostExecutionHook(hkv)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run clasHooks.ipynb
try:
	#transformationPreExecutionHook()

	clasautofe = TransformationMain.run(hkv,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": "listing_id", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "26993846.43", "stddev": "12318612.41", "min": "274589", "max": "48220707", "missing": "0"}, "updatedLabel": "listing_id"}, {"transformationsData": [{"feature_label": "name", "transformation_label": "String Indexer"}], "feature": "name", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "123123.0", "stddev": "nan", "min": "!5 min frm MTR!Cozy 2 BDR @ the heart of KLN - TST", "max": "\u00ef\u00bc\u0081NEW\u00ef\u00bc\u0081 #STUDIO@JORDAN#3MINS MTR#8MINS AIRPORT TRAIN", "missing": "0", "distinct": "496"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "name"}, {"transformationsData": [{"feature_label": "host_id", "transformation_label": "String Indexer"}], "feature": "host_id", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "95783051.5", "stddev": "91552205.01", "min": "101777143", "max": "98992897", "missing": "0", "distinct": "350"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "host_id"}, {"transformationsData": [{"feature_label": "host_since", "transformation_label": "String Indexer"}], "feature": "host_since", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "42429.52", "stddev": "782.94", "min": "40222", "max": "44189", "missing": "0", "distinct": "330"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "host_since"}, {"transformationsData": [{"feature_label": "host_response_time", "transformation_label": "String Indexer"}], "feature": "host_response_time", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "382", "mean": "", "stddev": "", "min": "a few days or more", "max": "within an hour", "missing": "118", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "host_response_time"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "host_response_rate", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "382", "mean": "0.72", "stddev": "0.36", "min": "0.0", "max": "1.0", "missing": "118"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "host_response_rate"}, {"transformationsData": [{"feature_label": "host_acceptance_rate", "transformation_label": "String Indexer"}], "feature": "host_acceptance_rate", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "342", "mean": "0.7", "stddev": "0.36", "min": "0", "max": "1", "missing": "158", "distinct": "47"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "host_acceptance_rate"}, {"transformationsData": [{"feature_label": "host_is_superhost", "transformation_label": "String Indexer"}], "feature": "host_is_superhost", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "f", "max": "t", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "host_is_superhost"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "host_total_listings_count", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "38.0", "stddev": "97.73", "min": "0.0", "max": "442.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "host_total_listings_count"}, {"transformationsData": [{"feature_label": "host_has_profile_pic", "transformation_label": "String Indexer"}], "feature": "host_has_profile_pic", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "f", "max": "t", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "host_has_profile_pic"}, {"transformationsData": [{"feature_label": "host_identity_verified", "transformation_label": "String Indexer"}], "feature": "host_identity_verified", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "f", "max": "t", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "host_identity_verified"}, {"transformationsData": [{"feature_label": "neighbourhood", "transformation_label": "String Indexer"}], "feature": "neighbourhood", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Central & Western", "max": "Yuen Long", "missing": "0", "distinct": "17"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "neighbourhood"}, {"transformationsData": [{"feature_label": "district", "transformation_label": "String Indexer"}], "feature": "district", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "0", "mean": "", "stddev": "", "min": "", "max": "", "missing": "500", "distinct": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "district"}, {"transformationsData": [{"feature_label": "city", "transformation_label": "String Indexer"}], "feature": "city", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Hong Kong", "max": "Hong Kong", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "city"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "latitude", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "22.3", "stddev": "0.05", "min": "22.20447", "max": "22.54014", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "latitude"}, {"transformationsData": [{"feature_label": "longitude", "transformation_label": "String Indexer"}], "feature": "longitude", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "114.15", "stddev": "0.05", "min": "113.91289", "max": "114.27257", "missing": "0", "distinct": "469"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "longitude"}, {"transformationsData": [{"feature_label": "property_type", "transformation_label": "String Indexer"}], "feature": "property_type", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Entire apartment", "max": "Entire apartment", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "property_type"}, {"transformationsData": [{"feature_label": "room_type", "transformation_label": "String Indexer"}], "feature": "room_type", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Entire place", "max": "Entire place", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "room_type"}, {"transformationsData": [{"feature_label": "accommodates", "transformation_label": "String Indexer"}], "feature": "accommodates", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "3.51", "stddev": "2.41", "min": "1", "max": "9", "missing": "0", "distinct": "15"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "accommodates"}, {"transformationsData": [{"feature_label": "bedrooms", "transformation_label": "String Indexer"}], "feature": "bedrooms", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "313", "mean": "1.73", "stddev": "0.98", "min": "1", "max": "5", "missing": "187", "distinct": "6"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "bedrooms"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "price", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1123.77", "stddev": "1520.57", "min": "80", "max": "10768", "missing": "0"}, "updatedLabel": "price"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "minimum_nights", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "12.55", "stddev": "29.08", "min": "1", "max": "365", "missing": "0"}, "updatedLabel": "minimum_nights"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "maximum_nights", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1125.0", "stddev": "0.0", "min": "1125", "max": "1125", "missing": "0"}, "updatedLabel": "maximum_nights"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "review_scores_rating", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "331", "mean": "90.93", "stddev": "12.06", "min": "20", "max": "100", "missing": "169"}, "updatedLabel": "review_scores_rating"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "review_scores_accuracy", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "331", "mean": "9.34", "stddev": "1.19", "min": "2", "max": "10", "missing": "169"}, "updatedLabel": "review_scores_accuracy"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "review_scores_cleanliness", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "331", "mean": "9.05", "stddev": "1.29", "min": "2", "max": "10", "missing": "169"}, "updatedLabel": "review_scores_cleanliness"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "review_scores_checkin", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "331", "mean": "9.54", "stddev": "1.07", "min": "2", "max": "10", "missing": "169"}, "updatedLabel": "review_scores_checkin"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "review_scores_communication", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "331", "mean": "9.51", "stddev": "1.16", "min": "2", "max": "10", "missing": "169"}, "updatedLabel": "review_scores_communicati..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "review_scores_location", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "331", "mean": "9.6", "stddev": "0.94", "min": "2", "max": "10", "missing": "169"}, "updatedLabel": "review_scores_location"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "review_scores_value", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "331", "mean": "9.02", "stddev": "1.18", "min": "2", "max": "10", "missing": "169"}, "updatedLabel": "review_scores_value"}, {"transformationsData": [{"feature_label": "instant_bookable", "transformation_label": "String Indexer"}], "feature": "instant_bookable", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "f", "max": "t", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "instant_bookable"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "SUM", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "97.31", "stddev": "71.1", "min": "0", "max": "160", "missing": "0"}, "updatedLabel": "SUM"}, {"transformationsData": [{"feature_label": "Years until 2021", "transformation_label": "String Indexer"}], "feature": "Years until 2021", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "5.35", "stddev": "2.19", "min": "1", "max": "9", "missing": "0", "distinct": "11"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Years until 2021"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Recommendation factor", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "547.13", "stddev": "473.75", "min": "0", "max": "1628", "missing": "0"}, "updatedLabel": "Recommendation factor"}, {"transformationsData": [{"feature_label": "Recommended Properties", "transformation_label": "String Indexer"}], "feature": "Recommended Properties", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Average", "max": "NotRecommended", "missing": "0", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Recommended Properties"}]}))

	#transformationPostExecutionHook(clasautofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run clasHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionClassification(clasautofe, ["listing_id", "host_response_rate", "host_total_listings_count", "latitude", "price", "minimum_nights", "maximum_nights", "review_scores_rating", "review_scores_accuracy", "review_scores_cleanliness", "review_scores_checkin", "review_scores_communication", "review_scores_location", "review_scores_value", "SUM", "Recommendation factor", "name_stringindexer", "host_id_stringindexer", "host_since_stringindexer", "host_response_time_stringindexer", "host_acceptance_rate_stringindexer", "host_is_superhost_stringindexer", "host_has_profile_pic_stringindexer", "host_identity_verified_stringindexer", "neighbourhood_stringindexer", "district_stringindexer", "city_stringindexer", "longitude_stringindexer", "property_type_stringindexer", "room_type_stringindexer", "accommodates_stringindexer", "bedrooms_stringindexer", "instant_bookable_stringindexer", "Years until 2021_stringindexer"], "Recommended Properties_stringindexer")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    Accuracy = np.round((100 * sklearn.metrics.accuracy_score(y_true=y_test, y_pred=y_predicted)), 1)
    F1= np.round(
            (100 * sklearn.metrics.f1_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Precision= np.round((
                100 * sklearn.metrics.precision_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Recall = np.round((
                100 * sklearn.metrics.recall_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    display(" Accuracy of Prediction on test data    : %s"%Accuracy)
    display(" F1 score of Prediction on test data    : %s"%F1)
    display(" Precision of Prediction on test data   : %s"%Precision)
    display(" Recall of Prediction on test data      : %s"%Recall)
    display(df.head())
except Exception as ex:
    logging.error(ex)

spark.stop()

